In [1]:
from init import *

In [2]:
model = sdk.Model()
model.TimeStart = 0
model.TimeSteps = 10
model.NumPaths = 1
model.Add(sdk.IndependentGaussian())
TimeHorizon = 10

In [3]:
ra = 0.05
rb = 0.05
r = lambda t: ra+rb*t
interpolation_points = 1000
dt = (TimeHorizon-model.TimeStart)/interpolation_points

process = model.Add(sdk.Linear1DInterpolation(
    ref = -1, # the function argument is time: f(t). In this case we need to use ref=-1
    xmin       = model.TimeStart,
    xmax       = model.TimeStart+dt*interpolation_points,
    y          = [r(i*dt) for i in range(interpolation_points)],
    title      = "short rate"
))


In [4]:
t_grid_points = 4
dt = (TimeHorizon-model.TimeStart)/t_grid_points
t_grid = [model.TimeStart+dt*i for i in range(t_grid_points+1)]
sum_process = model.Add(sdk.SumAtPoints(
    underlying = process.GetStateNumber(),
    t = t_grid[1:],
    title = "SumAtPoints"
))
print(t_grid)

[0.0, 2.5, 5.0, 7.5, 10.0]


In [5]:
# t_grid_points = 4
# dt = (TimeHorizon-model.TimeStart)/t_grid_points
# t_grid = [model.TimeStart+dt*i for i in range(t_grid_points+1)]
average_process = model.Add(sdk.AverageInInterval(
    underlying = process.GetStateNumber(),
    t = t_grid,
    title = "Average"
))
print(t_grid)

[0.0, 2.5, 5.0, 7.5, 10.0]


In [6]:
for t in t_grid:
    print(t)
    model.evaluations.append(sdk.EvaluationPoint(0,t))

0.0
2.5
5.0
7.5
10.0


In [7]:
results = sdk.run (model, server)
results.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,Linear1DInterpolation,short rate,0,0.0,0,1,0.105500,None,None,None
1,SumAtPoints,SumAtPoints,1,0.0,0,1,0.000000,None,None,None
2,AverageInInterval,Average,2,0.0,0,1,0.000000,None,None,None
3,Linear1DInterpolation,short rate,0,2.5,2,1,0.216500,None,None,None
4,SumAtPoints,SumAtPoints,1,2.5,2,1,0.216500,None,None,None
5,AverageInInterval,Average,2,2.5,2,1,0.161000,None,None,None
6,Linear1DInterpolation,short rate,0,5.0,4,1,0.327500,None,None,None
7,SumAtPoints,SumAtPoints,1,5.0,4,1,0.544000,None,None,None
8,AverageInInterval,Average,2,5.0,4,1,0.272000,None,None,None
9,Linear1DInterpolation,short rate,0,7.5,6,1,0.438500,None,None,None


$$
\sum_{i=1}^m P(t_0,t_i) N_i
$$
- $t_0$ is the _current_ time poiunt
- $m$ the number of time intrervals of the leg ($m>0$)
- $N_i$ the notional value at time point $t_i$

In [8]:
# short_rate_state = 0
# short_rate_sum   
# results.GetStateEvaluationResult(0,2).mean